In [3]:
import pandas as pd
import glob

## Exploracion errores en los archivos
* Exploraremos los datos y sus headers, verificando que todos esten bien identificados y que las tablas sean todas rectangulares, sin datos perdidos

In [3]:
headers = ['Codigo', 'Fecha', 'Hora', 'Temperatura', 'Humedad',
           'Direccion de Viento', 'Velocidad de Viento', 'Precipitacion',
           'Radiacion Solar', 'Presion Atmosferica']

def bad_headers(df):
    return any([name not in headers for name in df.columns]) or len(headers) != len(df.columns)

def has_nan(df):
    return df.isnull().any().any()

def find_bad_excels(region, print_all=False):
    counter_bads = 0
    counter = 0
    print("===> ", region)
    for folder in glob.glob("./PAULA/{}/*".format(region)):
        for file in glob.glob(folder + "/*.xls"):
            counter += 1
            try:
                df = pd.read_excel(file)
                if bad_headers(df):
                    print("ERROR.badHeader.", file)
                    print(df.count())
                    counter_bads += 1
                elif has_nan(df):
                    print("ERROR.NaN.", file)
                    print(df.count())
                    counter_bads += 1
                else:
                    if print_all:
                        print("GOOD.{}".format(file))
            except:
                counter_bads += 1
                print("ERROR.file: {}".format(file))
    return counter, counter_bads

In [4]:
counter = 0
counter_bads = 0
for region in glob.glob("./PAULA/*"):
    c, c_b = find_bad_excels(region.split("\\")[-1])
    counter += c
    counter_bads += c_b
print("Total de archivos:", counter)
print("Total de archivos con problemas:", counter_bads)

===>  ARICA Y PARINACOTA
ERROR.file: ./PAULA/ARICA Y PARINACOTA\SOCOROMA\Socoroma. 01-01-09-31-06-2009.xls
ERROR.file: ./PAULA/ARICA Y PARINACOTA\SOCOROMA\Socoroma. 01-07-09 31-12-09.xls
ERROR.file: ./PAULA/ARICA Y PARINACOTA\TRUFA AZAPA MEDIO\TrufaAzapaMedio.01-01-2009-30-06-2009.xls
ERROR.file: ./PAULA/ARICA Y PARINACOTA\TRUFA AZAPA MEDIO\TrufaAzapaMedio.01-07-2009-31-12-2009.xls
===>  ATACAMA
ERROR.file: ./PAULA/ATACAMA\BODEGA\Bodega01-07-2009-31-12-2009.xls
ERROR.file: ./PAULA/ATACAMA\COPIAPO\Copiapo01-07-2009-31-12-2009.xls
ERROR.file: ./PAULA/ATACAMA\TRANQUE-LAUTARO\Tranquelautaro01-06-2009-30-06-2009.xls
===>  AYSEN DEL GENERAL DEL C.I.CAMPO
===>  COQUIMBO
ERROR.file: ./PAULA/COQUIMBO\CARACHILLA\Carachilla01-01-2009-30-06-2009.xls
ERROR.file: ./PAULA/COQUIMBO\ILLAPEL\Illapel01-01-2009-30-06-2009.xls
===>  DE LA ARAUCANIA
ERROR.NaN. ./PAULA/DE LA ARAUCANIA\CUNCO\Cunco-01-07-2009-31-12-2009.xls
Codigo                 17363
Fecha                  17363
Hora                   17363


Total de archivos: 509
Total de archivos con problemas: 32


### Posibles correcciones
* Para las tablas con malos headers basta con renombrarlos y como son pocos esto se puede realizar manual
* Para las tablas con NaN, tenemos 2 opciones:
    * OPCION 1: borrar la fila especifica que contiene el dato NaN, esto puede causar series de tiempo irregulares
    * OPCION 2: interpolar el dato perdido (lo mas facil seria una interponacion lineal entre el dato anterior y el siguiente, si es el ultimo borrarlo)

##### por ahora es recomendable evitar estos excels para que realicen sus exploraciones

In [7]:
df = pd.read_excel("./PAULA/VALPARAISO/PETORCA/Petorca-01-07-2009-31-12-2009.xls")
df.head()

,Codigo,Fecha,Hora,Temperatura,Humedad,Direccion de Viento,Velocidad de Viento,Precipitacion,Radiacion Solar,Presion Atmosferica,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,320025,2009-07-01,00:00:00,8.5,96.1,219,0.2,0.0,1,993.5,NaN,NaN,NaN
1,320025,2009-07-01,00:15:00,8.5,96.3,237,0.4,0.0,1,993.8,NaN,NaN,NaN
2,320025,2009-07-01,00:30:00,8.5,96.5,21,0.4,0.0,1,993.6,NaN,NaN,NaN
3,320025,2009-07-01,00:45:00,8.3,96.5,102,0.2,0.0,1,993.6,NaN,NaN,NaN
4,320025,2009-07-01,01:00:00,8.2,96.6,108,0.2,0.0,1,993.5,NaN,NaN,NaN


In [9]:
df = pd.read_excel("./PAULA/DE LA ARAUCANIA/CUNCO/Cunco-01-07-2009-31-12-2009.xls")
df.count()

Codigo                 17363
Fecha                  17363
Hora                   17363
Temperatura            17363
Humedad                17363
Direccion de Viento    17362
Velocidad de Viento    17363
Precipitacion          17363
Radiacion Solar        17363
Presion Atmosferica    17363
dtype: int64

In [15]:
df[df.isnull().any(axis=1)]

,Codigo,Fecha,Hora,Temperatura,Humedad,Direccion de Viento,Velocidad de Viento,Precipitacion,Radiacion Solar,Presion Atmosferica
1193,380090,2009-07-13,10:15:00,2.3,99,NaN,0.5,8.3,216,1031.7


In [19]:
df.iloc[1191:1196]

,Codigo,Fecha,Hora,Temperatura,Humedad,Direccion de Viento,Velocidad de Viento,Precipitacion,Radiacion Solar,Presion Atmosferica
1191,380090,2009-07-13,09:45:00,0.8,97,2.25,0.5,0.2,163,1031.2
1192,380090,2009-07-13,10:00:00,1.4,97,2.25,0.5,19.8,188,1031.7
1193,380090,2009-07-13,10:15:00,2.3,99,NaN,0.5,8.3,216,1031.7
1194,380090,2009-07-13,10:30:00,3.6,98,9.00,0.5,6.8,248,1031.7
1195,380090,2009-07-13,10:45:00,4.6,96,20.25,0.9,5.8,275,1031.8
